In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
epsilon = 0.2
mu = 0.1
delta_x = 0.4
delta_t = 0.1
N_x = 130
L = N_x*delta_x


In [ ]:
u_0 = np.zeros(N_x)
for j in range(len(u_0)):
    u_0[j] = 0.5*(1-np.tanh((j-25)/5))

def Koorteweg_de_Vries(num_timesteps, u_0):
    u = [u_0]
    for t in range(num_timesteps):
        u_prev = u[-2] #n-1
        u_current = u[-1] #n
        u_next = np.zeros(N_x)
        for j in range(N_x):
            u_next[j] = u_prev[j] - (epsilon/3)*(delta_t/delta_x)*(u_current[j+1]+u_current[j]+u_current[j-1])*(u_current[j+1]+u_current[j-1])-mu*(delta_t / (delta_x**3))*(u_current[j+2]+2*u_current[j-1]-2*u_current[j+1]-u_current[j-2])
        u.append(u_next)